In [71]:
import pandas as pd
import numpy as np

In [72]:
# Colab Imports
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [113]:
def df_load_merge_pipeline(file_ids_0, year, file_id_1):
  '''
  Reads in CSVs uploaded to drive from CA.gov Department of Water Resources site: https://cimis.water.ca.gov/Default.aspx
  Should be approximately 4x csvs per year (can only download in batches).
  Constructs file names based off a naming convention.

  After cleaning, merges CIMIS csvs with Wikipedia csv.

    Note: 2020 wikipedia table needed station name cleaning

  PARAMETERS
  ----------
    file_ids_0: list of string of Drive file ids representing CIMIS csvs
    year: string re: '2018', '2019', or '2020'
    file_id_1: Wikipedia csv for days with fires in year

  RETURNS
  -------
    df: cleaned/merged dataframe for the year
  '''
  # First read in and combine all CIMIS csvs
  months = [[1, 3], [4, 6], [7, 9], [10, 12]]
    # Created in September 2020
  months2020 = [[1, 3], [4, 6], [7, 9]]
  all_dfs = []
  
  if year == '2020':
    for i, (file_id, month_pair) in enumerate(zip(file_ids_0, months2020)):
      create_file = drive.CreateFile({'id': file_id})

      file_name = f'{month_pair[0]}_{year}_{month_pair[1]}_{year}.csv'
      create_file.GetContentFile(file_name)
      dfi = pd.read_csv(file_name)
        
      all_dfs.append(dfi)

  else:
    for i, (file_id, month_pair) in enumerate(zip(file_ids_0, months)):
      create_file = drive.CreateFile({'id': file_id})

      file_name = f'{month_pair[0]}_{year}_{month_pair[1]}_{year}.csv'
      create_file.GetContentFile(file_name)
      dfi = pd.read_csv(file_name)
        
      all_dfs.append(dfi)
  
  df = pd.concat(all_dfs)

  cols = ['Jul', 'qc', 'qc.1', 'qc.2', 'qc.3', 'qc.4', 'qc.5', 'qc.6', 'qc.7', 'qc.8', 'qc.9', 'qc.10', 'qc.11', 'qc.12', 'qc.13']
  df.drop(cols, axis=1, inplace=True)

  # Second read in and clean Wikipedia csv 
  fires = drive.CreateFile({'id': file_id_1})
  wiki_file_name = f'{year}_fires.csv'
  fires.GetContentFile(wiki_file_name)
  fires = pd.read_csv(wiki_file_name)

  # Cleaning
  date_cols = ['Start date', 'Containment date']
  if year == '2018':
    for col in date_cols:
      fires[col] = pd.to_datetime(fires[col], format='%B %d, %Y')
      fires[col] = fires[col].apply(lambda x: x.strftime('%-m/%-d/%Y'))
  else:
    for col in date_cols:
      fires[col] = pd.to_datetime(fires[col], format='%B %d')
      fires[col] = fires[col].apply(lambda x: x.strftime(f'%-m/%-d/{year}'))

  fires[date_cols] = fires[date_cols].apply(pd.to_datetime)
  cleaner_fires = fires.loc[fires.index.repeat((fires[date_cols[1]] - fires[date_cols[0]]).dt.days + 1)]
  cleaner_fires['Date'] = cleaner_fires[date_cols[0]] + pd.to_timedelta(cleaner_fires.groupby(level=0).cumcount(), unit='d')
  cleaner_fires = cleaner_fires.reset_index(drop=True)
  cleaner_fires.drop(date_cols, axis=1, inplace=True)
  cleaner_fires['Date'] = pd.to_datetime(cleaner_fires['Date'], format='%B %d, %Y')
  cleaner_fires['Date'] = cleaner_fires['Date'].apply(lambda x: x.strftime('%-m/%-d/%Y'))

  # Cleaning 2020 stations
  if year == '2020':
    # Load active stations list from CIMIS csv
    _ = drive.CreateFile({'id': '1q8MCC8ebs29LmGj6-K-S8_PXgsvLPSlc'})
    _.GetContentFile('cimis_station_list.csv')  
    stations = pd.read_csv('cimis_station_list.csv')
    stations = stations.rename(columns={'Fresno/F.S.U. USDA': 'Stn Name', 'Fresno': 'County', 'Inactive': 'Status'})
    stations.drop('1', axis=1, inplace=True)
    stations = stations[stations['Status'] == 'Active']

    cleaner_fires = cleaner_fires.merge(stations, how='left', on=['County'])
    cleaner_fires['Stn Name'] = cleaner_fires['Stn Name_y']
    cleaner_fires.drop(['Stn Name_x', 'Status', 'Stn Name_y'], axis=1, inplace=True)
    cleaner_fires = cleaner_fires.dropna()

  # Third add target columns and merge df with fires
  df['Target'] = 0
  cleaner_fires['Target'] = 1

  full_df = df.merge(cleaner_fires, how='left', on=['Stn Name', 'Date'])
  full_df['Target'] = np.where(full_df['Target_y'] == 1.0, 1, 0)
  full_df.drop(['Target_x', 'County', 'Target_y'], axis=1, inplace=True)

  if year != '2018':
    full_df.drop('Acres', axis=1, inplace=True)

  return full_df

In [118]:
all_years = []

In [98]:
ids_2018 = ['1W45veu7NEtmcqmuJjXmM_SfvEvXrTKgM', '1S1t6WNuHO6BkCFROko2QawIaJnwpGBWT', '1wmr7tbXqQAMhEZLvmm67inu7FalvPMPX', '17ZyxEAYLeO7G1urypi5c_p0oHfdYA7Qv']
wiki_id_2018 = '1MUFRzAX-RmO_tiyjae2cw7yZ4w-ioauL'
df_2018 = df_load_merge_pipeline(ids_2018, '2018', wiki_id_2018)
all_years.append(df_2018)

In [99]:
ids_2019 = ['12qpKaO3_9ObFUgB0LAaVUBVgrEwETurY', '1fFXiA52uyzNqyvlnDVmr1Tx5ZIXDl2f6', '1d-QUU8zc2CirTbYf5uiXAE-iTtIIngMg', '13s_LyW8plqBISXq1HTS26F7mFGBBzBas']
wiki_id_2019 = '1IBDg9Ro-3Bk8XpYVdC6JLlofYjMjHb0f'
df_2019 = df_load_merge_pipeline(ids_2019, '2019', wiki_id_2019)
all_years.append(df_2019)

In [114]:
ids_2020 = ['1Wkq1-2SLML4EgnC_phMqqsKRzmv52ubO', '10Ikt-QDEVX2Cg463kG8UZJ62-1dlyRS8', '1Ezs6CIoTL4IZ0UDKuaVEiZtpf3h9Hc5t']
wiki_id_2020 = '1OPJxlzRFpqKLomdWs2nmL8MIJrZA2eZs'
df_2020 = df_load_merge_pipeline(ids_2020, '2020', wiki_id_2020)
all_years.append(df_2020)

In [122]:
df = pd.concat(all_years, ignore_index=True)

In [124]:
df.Target.value_counts()

0    144585
1      4987
Name: Target, dtype: int64

In [125]:
df.to_csv('conditions_df.csv', index=False)

In [126]:
test = pd.read_csv('conditions_df.csv')

In [127]:
test

,Stn Id,Stn Name,CIMIS Region,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),Notes,Target
0,2,FivePoints,San Joaquin Valley,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1,NaN,0
1,2,FivePoints,San Joaquin Valley,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,74.5,51.3,NaN,0
2,2,FivePoints,San Joaquin Valley,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,107.5,51.3,NaN,0
3,2,FivePoints,San Joaquin Valley,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,140.2,53.0,NaN,0
4,2,FivePoints,San Joaquin Valley,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,101.4,54.4,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149567,262,Linden,San Joaquin Valley,9/9/2020,0.11,0.00,306.0,14.0,82.5,53.2,64.9,95.0,33.0,67.0,53.6,2.6,63.1,70.6,NaN,0
149568,262,Linden,San Joaquin Valley,9/10/2020,0.07,0.00,245.0,15.4,77.1,52.4,63.5,96.0,52.0,77.0,56.2,2.3,54.8,69.7,NaN,0
149569,262,Linden,San Joaquin Valley,9/11/2020,0.13,0.00,412.0,15.8,85.1,50.6,65.4,97.0,39.0,74.0,56.9,2.1,49.8,69.4,NaN,0
149570,262,Linden,San Joaquin Valley,9/12/2020,0.10,0.00,338.0,16.7,83.2,50.6,65.0,97.0,46.0,79.0,58.4,1.6,38.8,69.2,NaN,0
